# How to run IDyOMpy and codeForPaper



#### Create a new conda environment and activate it

(the current one is saved in SigMA-IDyOMpy/codeForPaper-IDyOMpy-/IDyOM/miniconda3/envs so navigate there first and then activate)

In [ ]:
conda create -n idyompyenv
conda activate idyompyenv

#### Get the code_for_paper repo from GitHub

In [ ]:
git clone https://github.com/GuiMarion/codeForPaper-IDyOMpy-.git

#### Navigate inside the "codeForPaper-IDyOMpy" and clone the original IDyOM repo and navigate inside:

In [ ]:
cd codeForPaper-IDyOMpy
git clone https://github.com/GuiMarion/IDyOM.git
cd IDyOM

#### Make a virtual environment 

In [ ]:
python3 -m venv venv
source venv/bin/activate

#### Install the dependencies:

In [ ]:
pip install -r requirements.txt

## Run Bash Scripts for Benchmark plots

#### Copy over the relevant bash script inside of the IDyOM folder. Then make a text file to save the terminal output into so you can check for errors later. (I named mine "bm_IDyOMpy_PPM.txt")

#### From inside IDyOMpy, make the bash script executable (replace bash script name as needed)

In [ ]:
chmod +x run_for_benchmark_IDyOMpy.sh

#### Make sure your virtual environments are activated, and then run the bash script

In [ ]:
./run_for_benchmark_IDyOMpy.sh 2>&1 | tee bm_IDyOMpy_PPM.txt
./run_for_benchmark_IDyOMpy_ppm.sh 2>&1 | tee bm_IDyOMpy_PPM.log

run_for_benchmark_IDyOMpy
- started at 3:31 pm Dec 10th
- finished at 3:49 pm Dec 10th

run_for_benchmark_IDyOMpy_PPM (run consecutively)
- started at 3:52 pm Dec 10th 
- early termination
- results saved in bm_IDyOMpy_PPM.txt

run_for_benchmark_IDyOMpy_Genuine (run after re-cloning a new repo from scratch)
- started at 5:05 pm Dec 10th
- early termination at 7:05 pm Dec 10th 
- results saved in benchmark_GE.txt

## Use the python versions of the Bash Scripts

To use the python version of the scripts, navigate your terminal to the IDyOM folder, then call this code

In [ ]:
python3 run_for_benchmark_IDyOM.py | tee benchmark.log

python3 run_for_benchmark_IDyOMpy_genuine.py | tee benchmark_genuine.log

python3 run_for_benchmark_IDyOMpy_PPM.py | tee benchmark_ppm.log

## If you need to restart 
delete out the models created 
delete out the "out" folder in IDyOM

### Notes on running:

If you need to restart, delete the IDyOM folder within the codeForPaper repo, delete the benchmark results folder as well, then restart from cloning in the new IDyOM repo. A folder called .TEMP and out are created during this Bash script. 

PPM model bash script checks for an additional 3 models (.._originalPPM) to be deleted if present.  

#### Running the actual python files


**IDyOM_AE** file runs App.py -t (TRAIN_FOLDER) and -s (TRIAL_FOLDER) what surprisal is calculated on:
| -t (Train)             | -s (Calc Surprisal)    |
|------------------------|------------------------|
| train_shanxi (Chinese) | bach_Pearce            |
| bach_Pearce (Bach)     | train_shanxi           |
| mixed2 (Large Western) | stimuli/GregoireMcGill |


Then file computes cross entropy of the bach_Pearce folder while also running App.py on these combinations of training / surprisal validation:
| -t (Train)             | -s (Calc Surprisal)    |
|------------------------|------------------------|
| mixed2 (Large Western) | stimuli/giovanni /          |
| mixed2 (Large Western) | stimuli/Gold/        |
| bach_Pearce (Bach)  | stimuli/GregoireMcGill |
| bach_Pearce (Bach)  | stimuli/giovanni /          |


Also then calls -e on the bach_Pearson folder, which trains and evaluates over training, but does just -c cross entropy calculation for train_shanxi and mixed2. 

These scripts print and save alot of results, which are then copied from the "out" folder into the benchmark_results folder. 


**IDyOM_PPM** does a nearly similar set of actions, except it adds -o 1 to each line meaning to use the Partial Pattern Matching function from the original Lisp implementation. 

It also upfront starts to calc the cross entropy of all 3 datasets, bach_Pearce, train_shanxi, and mixed2 with -o 1 while training the same 4  combination of datasets and validations above. 

Once that's all done, it copies the values from out folder into the benchmark results

**IDyOM_GE** has the same exact set up but with -g instead of -o everywhere. This is noted to be 5 times slower than the original file which makes sense based on my latest experiments, but I havent been able to get the job to fully finish. 

## Looking at the Results

In the newly created "benchmark_results" folder located above the IDyOM folder, you will see the "likelihoods_cross_eval_k_fold_5_quantization_24_maxOrder_20_viewpoints_pitch_length" results in matplot and in pickle form for each of the datasets trained on.

In matplot, you will unpack multiple variables each corresponds to a song. For each song you have the Information Content (generalization error) as the first dimension, and then the Relative Entropy as the second dimension. They are both vectors over the time dimension. These measures relate to the surprisal we are seeing. 

## Open Questions:
- How do we group together results for the mean generalization error (IC) values in the chart? is this over time first in the song, then across songs?
- Why does IDyOM_GE and PPM not finish running but the other one does? 

## Things to try next:
- Comment out all versions of results we don't need and see if we can just run once on the dataset 
- Read the Bash script thoroughly and convert it to a customizable Python script including printing the terminal results into a text file (or add this to the bash code??)

- confirm that out put matches his output in main analysis (within 0.01 of the value, likely due to randomness)

- find scripts to generate figures in mainanalysis (found eps files, but not the function that goes from matlab to .eps
)
- confirm what the unit of time is (midi note onset, or time duration) - confirmed this is note onset, the number of instances is equal to the # of midi notes



## Convert .eps files to .pdf for viewing

In [ ]:
gs -dSAFER -dBATCH -dNOPAUSE \
   -sDEVICE=pdfwrite \
   -dEPSCrop=false \
   -sOutputFile=Fig1_GeneralizationErrors.pdf Fig1_GeneralizationErrors.eps

## Debugging PPM 

at line 351 is where the function breaks after printing "we saved the models" 
we are stuck at getSurpriseFromFolder line 513 of App.py 
try adding a tqdm bar loading bar to see what
because genuine entropies is false we are doing getLikelihoodfrom file ssimplifiled Entropies 

we added a break on the for loop line 538 in getsurprisefrom folder (after asking it to include a pbar)

Can this run faster on mps as device?

How to optimize a markov chain (bc its ancient)

in the markovChain class, the train method, maybe see how long training 1 single markovchain model takes, maybe do less. 

### NExt Stpes:
 - Run PPM over night with tee log and add the Pbar to the getSurpriseFromFolder function in the IDyOM class 
 - ask ChatGBT if we can make this run faster with mps?
 - Email Guilhem when it takes forever 
 - Message Iran 